<h1><span style="color:gray">ipyrad-analysis toolkit:</span> mrBayes</h1>

Bayesian phylogenetic inference can provide several advantages over ML approaches, particularly with regards to inferring dated trees (separating rate and time), and because they assess support differently, using a posterior sample of trees inferred from the full data set as opposed to bootstrap resampling of the data set. This may be particularly relevant to RAD-seq data that contains missing data. 

Bayesian inference programs often contain waaay too many options, which make them difficult to use. But it's kind of necessary in order to make informed decisions when setting priors on parameters, as opposed to treating the analysis like a black box. That being said, I've written a sort of blackbox tool for running mrbayes analyses. Once you've established a set of parameters that are best for your analysis this tool is useful to then automate it across many loci (e.g., see [`ipa.treeslider()`](https://ipyrad.readthedocs.io/en/latest/API-analysis/cookbook-treeslider.html)). 

### Required software

In [1]:
# conda install ipyrad -c bioconda
# conda install toytree -c eaton-lab
# conda install mrbayes -c bioconda -c conda-forge

In [2]:
import ipyrad.analysis as ipa
import toytree

In [3]:
# print current versions
print("toytree", toytree.__version__, '\n')
! mb -v

toytree 1.0.0 

MrBayes, Bayesian Analysis of Phylogeny

Version:   3.2.7a
Features:  SSE AVX FMA Beagle readline
Host type: x86_64-unknown-linux-gnu (CPU: x86_64)
Compiler:  gnu 7.3.0


### Setting mb param settings

The purpose of the ipyrad wrapper tool for mrbayes is merely for convenience. You should still take care to understand the priors and parameters that you are selecting by reading the mrbayes documentation. We only support two relatively simple models, a clock and non-clock model. Here I demonstrate the clock model with uncorrelated lognormal distributed rate variation and a birth-death tree prior. This run, which samples 500K iterations takes about 2 hours on a laptop. 

In [4]:
# the path to your NEXUS formatted file
nexfile = "/home/deren/Documents/ipyrad/sandbox/pedicularis/analysis-ipyrad/min10_outfiles/min10.nex"


In [5]:
# init mrbayes object with input data and (optional) parameter options
mb = ipa.mrbayes(
    name="pedicularis-min10",
    data=nexfile,
    clock=True, 
    ngen=500000,
    samplefreq=1000,
    nruns=1,
)

Existing results loaded for run [3], see .trees attribute.


In [6]:
mb.params.samplefreq = 5000
mb.params

brlenspr       clock:birthdeath    
clockratepr    lognorm(-7,0.6)     
clockvarpr     tk02                
extinctionpr   beta(2, 200)        
nchains        4                   
ngen           500000              
nruns          1                   
samplefreq     5000                
sampleprob     0.1                 
samplestrat    diversity           
speciationpr   exp(10)             
tk02varpr      exp(1.0)            
treeagepr      offsetexp(1, 5)     

In [7]:
# print the mb nexus string; this can be modified by changing .params
print(mb.nexus_string)

#NEXUS
execute /home/deren/Documents/ipyrad/sandbox/pedicularis/analysis-ipyrad/min10_outfiles/min10.nex;

begin mrbayes;
set autoclose=yes nowarn=yes;

lset nst=6 rates=gamma;

prset clockratepr=lognorm(-7,0.6);
prset clockvarpr=tk02;
prset tk02varpr=exp(1.0);
prset brlenspr=clock:birthdeath;
prset samplestrat=diversity;
prset sampleprob=0.1;
prset speciationpr=exp(10);
prset extinctionpr=beta(2, 200);
prset treeagepr=offsetexp(1,5);

mcmcp ngen=500000 nrun=1 nchains=4;
mcmcp relburnin=yes burninfrac=0.25;
mcmcp samplefreq=5000;
mcmcp printfreq=10000 diagnfr=5000;
mcmcp filename=/home/deren/Documents/ipyrad/newdocs/API-analysis/analysis-mb/pedicularis-min10.nex;
mcmc;

sump filename=/home/deren/Documents/ipyrad/newdocs/API-analysis/analysis-mb/pedicularis-min10.nex;
sumt filename=/home/deren/Documents/ipyrad/newdocs/API-analysis/analysis-mb/pedicularis-min10.nex;
end;



### Run mrbayes

This takes about 20 minutes to run on my laptop for a data set with 13 samples and ~1.2M SNPs and about 14% missing data. For a publication quality analyses you will likely want to run it quite a bit longer and to test for convergence of your runs using a tool like `tracer` and `treeannotator`.

In [ ]:
# run the command, (options: block until finishes; overwrite existing)
mb.run(block=True, force=True)

### Accessing results

You can access the results files of the mrbayes analysis from the mb analysis object from the `.trees` attribute, or of course you can also write out the full path to the result files which will be located in your working directory which can be set as a parameter option, otherwise is defaulted to `"./analysis-mb/".`

In [6]:
# you can access the tree results from the mb analysis object
mb.trees

constre   ~/Documents/ipyrad/newdocs/API-analysis/analysis-mb/pedicularis-min10.nex.con.tre
info      ~/Documents/ipyrad/newdocs/API-analysis/analysis-mb/pedicularis-min10.nex.lstat
posttre   ~/Documents/ipyrad/newdocs/API-analysis/analysis-mb/pedicularis-min10.nex.t

In [7]:
# for example to select the consensus tree path
mb.trees.constre

'/home/deren/Documents/ipyrad/newdocs/API-analysis/analysis-mb/pedicularis-min10.nex.con.tre'

### Plotting mb consesus tree
You can plot either the consensus tree (`.nex.cons.tre`) or the posterior distribution of trees (`.nex.t`) produced by a mrbayes run by loading the single tree or list of trees with `toytree` as demonstrated below. 

In [10]:
# load from the .trees attribute or from the saved tree file
tre = toytree.tree(mb.trees.constre, tree_format=10)

# draw the tree with styling
tre.draw(
    tip_labels_align=True, 
    scalebar=True,
    #node_labels="support",
);

<svg class="toyplot-canvas-Canvas" height="275.0px" id="te2664359f9b546b0ac7a16611c798c63" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 415.0 275.0" width="415.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 39618_rex_SRR1754723 38362_rex_SRR1754725 35236_rex_SRR1754731 40578_rex_SRR1754724 35855_rex_SRR1754726 30556_thamno_SRR1754720 33413_thamno_SRR1754728 41954_cyathophylloides_SRR1754721 41478_cyathophylloides_SRR1754722 30686_cyathophylla_SRR1754730 29154_superba_SRR1754715 33588_przewalskii_SRR1754727 32082_przewalskii_SRR1754729 0 4 7 11

### Plotting mb posterior distribution of trees

In [13]:
# load from the .trees attribute or from the saved tree file
mtre = toytree.mtree(mb.trees.posttre)

# remove the burnin manually
mtre.treelist = mtre.treelist[10:]

# draw a few trees on a shared axis
mtre.draw_tree_grid(
    nrows=1, ncols=4, start=20, shared_axis=True,
    height=400, 
);

<svg class="toyplot-canvas-Canvas" height="400.0px" id="tb4759a47e2d94494b1a0dd1563481031" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 700.0 400.0" width="700.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> 39618_rex_SRR1754723 38362_rex_SRR1754725 35236_rex_SRR1754731 35855_rex_SRR1754726 40578_rex_SRR1754724 30556_thamno_SRR1754720 33413_thamno_SRR1754728 29154_superba_SRR1754715 30686_cyathophylla_SRR1754730 41478_cyathophylloides_SRR1754722 41954_cyathophylloides_SRR1754721 33588_przewalskii_SRR1754727 32082_przewalskii_SRR1754729 38362_rex_SRR1754725 39618_rex_SRR1754723 35236_rex_SRR1754731 35855_rex_SRR1754726 40578_rex_SRR1754724 30556_thamno_SRR1754720 33413_thamno_SRR1754728 41954_cyathophylloides_SRR1754721 41478_cyathophylloides_SRR1754722 29154_superba_SRR1754715 30686_cyathophylla_SRR1754730 32082_przewalskii_SRR1754729 33588_przewalskii_SRR1754727 38362_rex_SRR1754725 39618_rex_SRR1754723 35236_rex_SRR1754731 35855_rex_SRR1754726 40578_rex_SRR1754724 30556_thamno_SRR1754720 33413_thamno_SRR1754728 41954_cyathophylloides_SRR1754721 41478_cyathophylloides_SRR1754722 29154_superba_SRR1754715 30686_cyathophylla_SRR1754730 32082_przewalskii_SRR1754729 33588_przewalskii_SRR1754727 39618_rex_SRR1754723 38362_rex_SRR1754725 35236_rex_SRR1754731 35855_rex_SRR1754726 40578_rex_SRR1754724 30556_thamno_SRR1754720 33413_thamno_SRR1754728 29154_superba_SRR1754715 30686_cyathophylla_SRR1754730 41478_cyathophylloides_SRR1754722 41954_cyathophylloides_SRR1754721 33588_przewalskii_SRR1754727 32082_przewalskii_SRR1754729 0.00 0.02 0.03

The command below plots a cloud tree diagram across the ~90 trees in the posterior distribution. You can see that the posterior has likely not converged on the true posterior yet, as the range of edge lengths does not look very well explored. This is something that should be checked with other software tools. 

In [18]:
# draw the posterior of trees overlapping
mtre.draw_cloud_tree(
    width=400, height=300,
    html=True,
    edge_colors="red",
    edge_style={"stroke-opacity": 0.02},
    tip_labels={i: i.rsplit("_", 1)[0] for i in mtre.treelist[0].get_tip_labels()},
);

<svg class="toyplot-canvas-Canvas" height="300.0px" id="tf39ef21bdea048ca909f36b0b64fb35c" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" viewBox="0 0 400.0 300.0" width="400.0px" xmlns="http://www.w3.org/2000/svg" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink"> <path d="M

### Save figures

In [19]:
import toyplot.pdf
canvas, axes = mtre.draw_cloud_tree(
    width=400, height=300,
    html=True,
    edge_colors="red",
    tip_labels={i: i.rsplit("_", 1)[0] for i in mtre.treelist[0].get_tip_labels()}
);
toyplot.pdf.render(canvas, "./pedicularis-min10-mb-tree.pdf")

### What else?

If you have reference mapped data then you should see the `.treeslider()` tool to infer trees in sliding windows along scaffolds; or the `.extractor()` tool to extract, filter, and concatenate RAD loci within a given window (e.g., near some known gene).